# Visualization of the dataset

The aim of this notebook is to display ten (10) choroplet maps to illustrate our dataset:


## 1) Libraries import and  dataset load and preparation

In [1]:
# Import common libraries
import pandas as pd
import numpy as np

# Import lib for Json and maps
import json
from shapely.geometry import shape, Point

# plotting lib
import branca.colormap as cm # for color steps
%matplotlib inline 
import folium
from IPython.display import display


# lib to convert GPS coordinates into a new format
from pyproj import Proj, transform

# lib to convert dates to datetime format
from datetime import datetime
from datetime import date, time
from dateutil.parser import parse

# Read the dataset
acc_data = '../_data/OTC_ACCIDENTS.csv'
compt_trafic_data = '../_data/OTC_COMPTAGE_TRAFIC.csv'
acc_df = pd.read_csv(acc_data, sep=';', encoding='latin-1')
compt_trafic_df = pd.read_csv(compt_trafic_data, sep=';', encoding='latin-1')

# Show the df to have a better idea
acc_df.head(3)

,ID_ACCIDENT,DATE_,GROUPE_ACCIDENT,CAUSE,COMMUNE,CONDITIONS_LUMINEUSES,CONDITIONS_METEO,CONSEQUENCES,COOR_X,COOR_Y,...,NB_MOTOS_50,NB_MOTOS_125,NB_MOTOS_11KW,NB_VOITURES_TOURISME,NB_VOITURES_LIVRAISON,NB_CAMIONS,NB_BUS,NB_TRAM,E,N
0,876245.0,2010-11-30 00:00:00,Dérapage ou perte de maîtrise,Inattention et distraction - Manque d'attentio...,Genève,Nuit,Chute de neige,Avec blessés légers,2500774.0,1117364.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2500774.0,1117364.0
1,879408.0,2010-12-08 00:00:00,Autres,Utilisation inadéquate du véhicule - Stationne...,Genève,Jour,Beau,Autres,2498974.0,1118100.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2498974.0,1118100.0
2,877254.0,2010-12-02 00:00:00,Dérapage ou perte de maîtrise,Inobservation de signaux ou de la signalisatio...,Vandoeuvres,Jour,Couvert,Avec blessés légers,2504618.0,1119635.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2504618.0,1119635.0


### Handling of Time data: convert them to datetime format
<a id='321Time'></a>

An important feature of the accidents datasets is the time: It answers the question when the accidents happen. This feature will be very useful for our further analysis. As time features, there are:
- Date
- Hour
- Day of the week

Let's check the format of this three features:

In [2]:
print(acc_df.DATE_.head(2))
print(acc_df.HEURE.head(2))
print(acc_df.JOUR.head(2))

0    2010-11-30 00:00:00
1    2010-12-08 00:00:00
Name: DATE_, dtype: object
0    1899-12-30 21:00:00
1    1899-12-30 14:00:00
Name: HEURE, dtype: object
0       Mardi
1    Mercredi
Name: JOUR, dtype: object


In order to use these features, we need to change the format. In addition, to enrich our analysis, we will create the `Year`, `Month` and `Day` of the month to find more correlations with the accidents:

In [3]:
# Datetime format
acc_df['DATE_'] = acc_df['DATE_'].apply(lambda d: pd.to_datetime(d))
acc_df['HEURE'] = acc_df['HEURE'].apply(lambda d: pd.to_datetime(d))

# Create new time features
acc_df['YEAR'] = [date.year for date in acc_df['DATE_']]
acc_df['MONTH'] = [date.month for date in acc_df['DATE_']]
acc_df['DAY'] = [date.day for date in acc_df['DATE_']]
acc_df['HEURE'] = acc_df['HEURE'].fillna(acc_df['HEURE'].iloc[0])  # Fillna with first value of the df (Error neglible)
#acc_df.info()

## converting GPS coordinates to standard epsg:4326 reference system

An other important question to ask is WHERE?. The localisation features will help to answer this question and find the relation with the other features. 
As localisation features, we can find:
- **COOR_X**: X coordenate in 'epsg_2056' reference system
- **COOR_Y**: X coordenate in 'epsg_2056' reference system

E and N gives the same information as COOR_X and COOR_Y. This is why they will be dropped. The COOR_X and COOR_Y coordenates will be projected in the GPS coordenates, also called 'epsg:4326' reference system. For this, the Proj and transform libraries will be used:

In [4]:
# projection definition
p1 = Proj(init='epsg:2056')
p2 = Proj(init='epsg:4326')

# Helper functions
def coord_proj(acc_df,i, p1, p2):
    x1 = acc_df['COOR_X'].loc[i]
    y1 = acc_df['COOR_Y'].loc[i]
    x2, y2 = transform(p1,p2,x1,y1)
    acc_df['COOR_X'].set_value(i, x2)
    acc_df['COOR_Y'].set_value(i, y2)
    return acc_df

# Project data
for i in range(0, len(acc_df['COOR_X'])-1):
    acc_df = coord_proj(acc_df,i, p1, p2)
# Delete unuseful columns
#del acc_df['N']
#del acc_df['E']
acc_df.head(3)

,ID_ACCIDENT,DATE_,GROUPE_ACCIDENT,CAUSE,COMMUNE,CONDITIONS_LUMINEUSES,CONDITIONS_METEO,CONSEQUENCES,COOR_X,COOR_Y,...,NB_VOITURES_TOURISME,NB_VOITURES_LIVRAISON,NB_CAMIONS,NB_BUS,NB_TRAM,E,N,YEAR,MONTH,DAY
0,876245.0,2010-11-30,Dérapage ou perte de maîtrise,Inattention et distraction - Manque d'attentio...,Genève,Nuit,Chute de neige,Avec blessés légers,6.153116,46.200401,...,1.0,0.0,0.0,0.0,0.0,2500774.0,1117364.0,2010,11,30
1,879408.0,2010-12-08,Autres,Utilisation inadéquate du véhicule - Stationne...,Genève,Jour,Beau,Autres,6.129641,46.206753,...,1.0,0.0,0.0,0.0,0.0,2498974.0,1118100.0,2010,12,8
2,877254.0,2010-12-02,Dérapage ou perte de maîtrise,Inobservation de signaux ou de la signalisatio...,Vandoeuvres,Jour,Couvert,Avec blessés légers,6.202445,46.221384,...,1.0,0.0,0.0,0.0,0.0,2504618.0,1119635.0,2010,12,2


# 2) VISUALIZATION of the dataset
Now that we finished the (brief) preprocessing of our dataset, let's go for the visualization of the dataset. To do so, we manually created a GeoJson file of different zones of the Genève canton thanks to the intuitive interface of the tool provided at http://geojson.io/#map=2/20.0/0.0. 

We divided the canton into 
* 10 polygons
* 8 regions

Observe that there are 2 less regions than polygon because, as displayed hereunder, the first three polygons in the top left of the map are considered belonging to the same region.


In [5]:
### HELPER FUNCTIONS TO PLOT THE DATA IN FANCY COLOURS

def serie_color(feature,serie_to_plot,colormap):
    """return color for countries, black if no data ':'
    depends on colormap settings"""

    if feature['id'] not in list(serie_to_plot.index):
        return '#000000'
    else:
        return colormap(serie_to_plot[feature['id']])
    
def create_colormap(index = 1, min_=0, max_=10,steps=10 ):
    if index == 1:
        step = cm.StepColormap(
            colors=['#FF0000','#FF8644','#0033FF','#60FFBF','#F6FF00'],
            vmin=min_, vmax=max_,
            caption='step'
            )
    if index == 2 :
        step = cm.linear.Set1.scale(min_, max_).to_step(steps)
    return step

In [6]:
### CREATING THE MAP
m = folium.Map(location=[46.254423, 6.142972], tiles='cartodbpositron', zoom_start=11)

### LOAD GeoJSON FILE CONTAINING POLYGONS
with open('map-3.geojson') as f:
    js = json.load(f)
folium.GeoJson(open("map-3.geojson",encoding = "utf-8-sig").read()).add_to(m)

### ADDING AN ID TO EVERY POLYGON
### we define a unique region between the three polygons in the top-left of the canton by setting the last
### two indices euqal again to 0
ids = [0,1,2,3,4,5,6,7,0,0]
for i in range(len(js['features'])):
    js['features'][i]['id']=ids[i]

### LET'S DISPLAY THE REGIONS:
m_serie = pd.Series(ids)
colormap = create_colormap(2,m_serie.min(),m_serie.max(),steps = 8) # create a colorfeature

    
folium.GeoJson(js,
                style_function=lambda feature: {
                'fillColor': serie_color(feature,m_serie,colormap),
                'color' : 'black',
                'weight' : 2,
                'dashArray' : '5, 5'
                }).add_to(m)
    
colormap.caption = 'Region IDs'
m.add_child(colormap)
    
m

## associating each accident to the correct polygon

For every accident in the acc_df dataframe, we have its GPS coordinates, but we don't know (yet) which region of the 8 defined above it belongs to. Let's add a column to the acc_df dataframe devoted to store the corresponding region for each accident: acc_df['RegionID']. 

The functions Point, shape of the library Shapely https://pypi.python.org/pypi/Shapely let us fill this column in a straightforward way in the few lines hereunder:

In [7]:
### ASSIGNING EACH ACCIDENT ITS CORRECT REGION

# adding a column in the dataframe devoted to store the corresponding region for each accident 
acc_df['Region ID'] = 0


polygons = js['features']
for i in acc_df.index:
    point = Point(acc_df.loc[i][['COOR_X','COOR_Y']])
    for feature in polygons:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            acc_df.set_value(i,'Region ID',feature['id'])
            
acc_df[['ID_ACCIDENT','COOR_X','COOR_Y','Region ID']].head()

,ID_ACCIDENT,COOR_X,COOR_Y,Region ID
0,876245.0,6.153116,46.200401,3
1,879408.0,6.129641,46.206753,3
2,877254.0,6.202445,46.221384,6
3,857129.0,6.180094,46.208173,6
4,843463.0,6.174721,46.199584,6


Now that we have all the geolocalization of the accidents, we can proceed to the visualization!
We'll split our visualization in 4 parts. Let's start with a first glimpse into the dataset, and let's display

 
## 2.1) HOW MANY ACCIDENTS HAPPENED PER REGION, PER YEAR

We import the widget interact becuase, as one can appreciate hereunder, **it let the user select the year from which extracting the corresponding data to plot**.

P.S. The color palette of all the plots is colorblind-friendly.

In [8]:
from IPython.html.widgets import interact

/Users/Mart/anaconda/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [9]:
def _maps(year):
    '''
    INPUT: year between 2010 and 2016
    OUTPUT: a choropleth map displaying the number of accidents occourred in each region in that year
    '''
    m = folium.Map(location=[46.254423, 6.142972], tiles='cartodbpositron', zoom_start=11)
    accidents_by_region = acc_df.loc[acc_df['YEAR']==year].groupby('Region ID').count()['ID_ACCIDENT']
    colormap = create_colormap(1,3,1350,steps = 8) # create a colorfeature
    folium.GeoJson(js,
                    style_function=lambda feature: {
                    'fillColor': serie_color(feature,accidents_by_region,colormap),
                    'color' : 'black',
                    'weight' : 2,
                    'dashArray' : '5, 5'
                    }).add_to(m)

    colormap.caption = 'Region IDs'
    m.add_child(colormap)
    return m

year_list = np.arange(2010,2017)
m = interact(_maps,year=year_list)
display(m)

<function __main__._maps>

By selecting different years, we can appreciate how the number of accidents significantly increased in the center of Geneva (from less that 1000 in 2010 to more than 1200 in 2016), while in the regions surrounding Genève it always stayed "low", that means under 170 accidents per year. 

Of course this is a naïve analysis, that doesn't normalize the increase and just takes the absolute number of accidents. It can be refined later if evaluated interesting. For now we content ourself of this first visualization.

## 2) VISUALIZE CATEGORICAL VARIABLES SUCH AS "GROUPE_ACCIDENT", "CONDITIONES_LUMINEUSES"

this kind of data will be represented in two ways, thanks again to the interact function. 
For every variable, a map has been created. In this map, the user can select 
* the year he wants i.e. "2015"
* the value of the variable he's interested in to visualize _i.e. "All the accidents that have GROUPE-ACCIDENT == accident par tamponament"_

Moreover, by selecting the box "normalize" in the widget displayed, the user can look at the same data in two different perspectives:

* visualize and compare between regions the absolute number of accidents that belong to a certain class, _i.e. "All the accidents that have GROUPE-ACCIDENT == accident par tamponament"_

* visualize the same data **as a percentage, normalized with respect to the total number of accidents that happened that year in that region. **

In [10]:
### ------------------- HELPER FUNCTIONS -------------------
### --------------------------------------------------------

def get_feature_to_plot(year,feature, value, normalize):
    '''
    INPUT:
    year: which year to look for in the dataframe acc_df
    feature: which feature to look for i.e. "GROUPE_ACCIDENT"
    value: the value of the feature to extract i.e. "accidents par tamponament"
    normalize: if True, normalize the data extracted as described above in the notebook
    '''
    serie = acc_df.loc[(acc_df['YEAR']==year) & (acc_df[feature]==value)].groupby('Region ID').count()['ID_ACCIDENT']
    if normalize == True:   
        total_number_of_accidents_in_year_per_region = acc_df.loc[(acc_df['YEAR']==year)].groupby('Region ID').count()['ID_ACCIDENT']
        serie = serie / total_number_of_accidents_in_year_per_region
    serie.fillna(0,inplace=True)
    print(serie)
    return serie


def create_map (year, feature, value, normalize):
    m_serie = get_feature_to_plot(year, feature, value, normalize) # get the serie
    colormap = create_colormap(1,m_serie.min(),m_serie.max(),steps = 5) # create a colorfeature
    m = folium.Map(location=[46.254423, 6.142972], tiles='cartodbpositron', zoom_start=11)
    
    folium.GeoJson(js,
                  style_function=lambda feature: {
                'fillColor': serie_color(feature,m_serie,colormap),
                'color' : 'black',
                'weight' : 2,
                'dashArray' : '5, 5'
                }).add_to(m)
    
    colormap.caption = 'Accidents'
    m.add_child(colormap)
   
    return m

year_list = np.arange(2010,2017)


def create_interactive_map(feature, normalize=True):
    value_list = list(acc_df[feature].value_counts().index)
    m = interact(create_map, year = year_list, feature = feature, value = value_list, normalize= normalize)
    return m

Let's plot the interactive map!

In [11]:
create_interactive_map('GROUPE_ACCIDENT')

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.create_map>

### What does this map mean? Let's take an example. 

In the map shown above, the user can access and display the data of all years from 2010 to 2017 relative to all different causes of accidents. 

**Let's select year 2015, select "accident par tamponament", and play with the "normalize" option**. 

The user will appreciate how 
* it's the center of Genève that has the highest number of accidents due to this cause (betwen 180 and 230) 

* "normalizing" (i.e. dividing each data by the total number of accidents occourred in 2015 in the corresponding regions) are the regions just outside Geneve that top the list. 

What does this mean? Although in the southern region of Lancy only 90-140 accidents "par tamponament" occour (grey color of the legend), they actually represent more than the 20% of the accidents in Lancy! In Geneve the percentage is actually lower, between 15% and 19%.

How this quantify the "risk" of a certain type of accident with respect to another, in different region of the canton? We still can't say: a deeper analysis has to be done, taking into account more variables (such as traffic, for example). 

### Let's do the same for also other, interesting categorical features of the dataset that caracterize an accident:

In [12]:
create_interactive_map('CONDITIONS_LUMINEUSES')

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.create_map>

Let's select **the year 2014, and take a look at the accidents happened at night.**
Thanks to this map we can appreciate how _in the western region of Satigny in 2014 happened only 8 accidents at nght, but they represent 30% of the total numebr of accidents! In the southern region of Avuly in 2014 happened  19 accidents at night, but they represent less than 22% of the accidents in this region._ 

In [13]:
create_interactive_map('CONSEQUENCES')

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.create_map>

From the map above we can notice a few interesting things:
* in each year, there are more accidents with deaths in the center of Genève than everywhere else
* in each year _normalizing_ we appreciate how are the outer regions that presents the highest percentage of accidents with deaths. This could suggest that in average, the accidents that occour far from the center are more serious that the ones happening in the city. 

This is easily understandable, if we only think that the average speed limit in the city is much lower that in outer roads, probably leading to this behaviour of the data.

## Let's now plot two features regarding the number of children involved in the accidents.

This time no normalization process can be done, since we don't have the data of how many people are involved in the accident in total. We'll just take a look at the raw data aggregated by year and region.

New helper functions are needed because of the particular interface of the command interact, that oblige us to define new helper functions to adapt to the new form of the dataset to plot (not normalized)

In [14]:
### ------------------- NEW HELPER FUNCTIONS -------------------
### ------------------------------------------------------------

def get_number_to_plot(year,feature):
    serie = acc_df.loc[(acc_df['YEAR']==year)].groupby('Region ID').sum()[feature]
    serie.fillna(0,inplace=True)
    return serie

def create_NB_map (year, feature):
    m_serie = get_number_to_plot(year, feature) # get the serie
    colormap = create_colormap(1,m_serie.min(),m_serie.max(),steps = 5) # create a colorfeature
    m = folium.Map(location=[46.254423, 6.142972], tiles='cartodbpositron', zoom_start=11)
    
    folium.GeoJson(js,
                  style_function=lambda feature: {
                'fillColor': serie_color(feature,m_serie,colormap),
                'color' : 'black',
                'weight' : 2,
                'dashArray' : '5, 5'
                }).add_to(m)
    
    colormap.caption = 'Accidents'
    m.add_child(colormap)
   
    return m



def create_interactive_ABSOLUTE_NB_map():
    year_list = np.arange(2010,2017)
    feature_list = ['NB_ENFANTS_IMPLIQUES', 'NB_ENFANTS_ECOLE']
    m = interact(create_NB_map, year = year_list, feature = feature_list)
    return m

In [15]:
create_interactive_ABSOLUTE_NB_map()

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.create_NB_map>

We can appreciate how amost all the accidents that involved a child occourred in the center of Genéve. To correctly evaluate the risk for children to be involved in accidents and not to jump to easy conclusions, other factors should to be taken into account, such the distribution of schools on the territory.

Let's move to the analysis of the two last aspects of our accidents: 
* the distribution of people injured in accidents
* the distribution of kind of vehicles involved in the accidents.

In [16]:
### let's define the features we are interested in:
blesses_et_tues = ['NB_BLESSES_LEGERS',
       'NB_BLESSES_GRAVES', 'NB_TUES']
vehicles = ['NB_BICYCLETTES',
       'NB_VAE_25', 'NB_VAE_45', 'NB_CYCLOMOTEURS', 'NB_MOTOS_50',
       'NB_MOTOS_125', 'NB_MOTOS_11KW', 'NB_VOITURES_TOURISME',
       'NB_VOITURES_LIVRAISON', 'NB_CAMIONS', 'NB_BUS', 'NB_TRAM']

###--------------------------------------------------------
###----------------- NEW HELPER FUNCTIONS -----------------
###--------------------------------------------------------

def get_number_to_plot_blesses_et_tues(year,feature,normalize=False):
    cum_df = acc_df.loc[(acc_df['YEAR']==year)].groupby('Region ID').sum()[blesses_et_tues]
    serie = cum_df[feature]
    if normalize==True:
        tot_nb = cum_df.sum(axis=1)
        serie  = serie/tot_nb
    serie.fillna(0,inplace=True)
    print(serie)
    return serie

def create_normalized_map_blesse_et_tues (year,feature,normalize=False):
    m_serie = get_number_to_plot_blesses_et_tues(year, feature,normalize) # get the serie
    colormap = create_colormap(1,m_serie.min(),m_serie.max(),steps = 5) # create a colorfeature
    m = folium.Map(location=[46.254423, 6.142972], tiles='cartodbpositron', zoom_start=11)
    
    folium.GeoJson(js,
                  style_function=lambda feature: {
                'fillColor': serie_color(feature,m_serie,colormap),
                'color' : 'black',
                'weight' : 2,
                'dashArray' : '5, 5'
                }).add_to(m)
    
    colormap.caption = 'Accidents'
    m.add_child(colormap)
   
    return m

def create_interactive_map_blesse_et_tues():
    year_list = np.arange(2010,2017)
    m = interact(create_normalized_map_blesse_et_tues, year = year_list, feature = blesses_et_tues,normalize=False)
    return m



def get_number_to_plot_vehicles(year,feature,normalize=False):
    cum_df = acc_df.loc[(acc_df['YEAR']==year)].groupby('Region ID').sum()[vehicles]
    serie = cum_df[feature]
    if normalize==True:
        tot_nb = cum_df.sum(axis=1)
        serie  = serie/tot_nb
    serie.fillna(0,inplace=True)
    print(serie)
    return serie

def create_normalized_map_vehicles (year,feature,normalize=False):
    m_serie = get_number_to_plot_vehicles(year, feature,normalize) # get the serie
    colormap = create_colormap(1,m_serie.min(),m_serie.max(),steps = 5) # create a colorfeature
    m = folium.Map(location=[46.254423, 6.142972], tiles='cartodbpositron', zoom_start=11)
    
    folium.GeoJson(js,
                  style_function=lambda feature: {
                'fillColor': serie_color(feature,m_serie,colormap),
                'color' : 'black',
                'weight' : 2,
                'dashArray' : '5, 5'
                }).add_to(m)
    
    colormap.caption = 'Accidents'
    m.add_child(colormap)
   
    return m

def create_interactive_map_vehicles():
    year_list = np.arange(2010,2017)
    m = interact(create_normalized_map_vehicles, year = year_list, feature = vehicles,normalize=False)
    return m

### INJURED PEOPLE

In [17]:
create_interactive_map_blesse_et_tues()

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.create_normalized_map_blesse_et_tues>

**Let's visualize the percentage of seriously injured peole "NB_BLESSES_GRAVE" and play around with the years:**_ we'll notice that the western region of Satigny has a lot more seriously injured people than all the others region!_ In 2011 the percentage was of 57%, while the second highest percentage of the other regions was only 37%! Also, in the years 2014-2015-2016 it was _by far_ the region with the highest percentage of seriously injured people, with really high percentages between 40% and 60%. This visualization suggest us to dig deeper into the kind of accidents happened in that region, to better understand why so many people are injured so badly. Probably the understanding of the kind of accidents typical of this region could help us. Once identified a possible cause, it could be possible to suggest specific actions that the commune could adopt to prevent such causes of serious accidents.

## VEHICLES INVOLVED IN THE ACCIDENT

In [18]:
create_interactive_map_vehicles()

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.create_normalized_map_vehicles>

From this map we see how plotting only the absolute numbers doesn't tell us much: the peak of accidents for every kind of vehicle is in the center of Geneve. However, as soon as we toggle the option "normalize" we see different patterns i.e. we see that the percentage of accidents involving bycicles is actually higher (for each year) in the regions outside the center.

# Further analysis and proposals for milestone 3
We saw that this kind of visualiztion is very useful to understand such a multivariate and complex dataset. For milestone three we aim to create a better, more syntetic, more flexible and more complete visualization at the same time. To reach such a goal, we think to:
* create a _unique_ interactive map similar to the ones presented here, capable of displaying all these data 
* implementing a more detailed and interactive subdivision between regions. It could be cool implementing different layers of geojson files diplaying more detailed regions as the user zoom in on the map to display more detailed subdivisions
* create a more synthetic dataframe from which taking the data to plot, with more significant variables extracted from other analysis
* implement new, more precise visualization method for accidents that display accidents as colored points on the map, letting the user precisely identify clusters of accidents on the map
* plotting accidents and traffic on the same map, so that an eventual (expected) correlation between the two can be visually analysed